# Observer

## O que é?

O _Observer_, também conhecido como _Publish-Subscribe_, estabelece uma relacao entre uma classe (Subject) e uma ou mais classes dependentes (Observers), de forma que todos os Observers sao informados de mudancas de estado no Subject.

## Por quê?

Ao permitir que diversos clientes sejam informados sobre as mudancas de estado, o padrao _Observer_ permite que a separacao entre a classe observada e a lógica de negócio dos clientes. Ele também pode ser uma forma elegante de reduzir a complexidade de um _Mediator_ -- nesse caso, as atualizacoes do Subject estabelecem relacoes indiretas entre diversos Observers.

## Estrutura



## Exemplo

Imaginemos uma casa com um termostato inteligente. O principal atrativo da nossa classe `SmartThermostat` é a possibilidade de executar uma série de funcoes diferentes, dependendo da temperatura. Numa implementacao bastante ingênua, teríamos o seguinte código:

In [ ]:
class AirConditioner:
    
    def __init__(self, on: bool = False):
        self.on = on
        
    def start(self, temperature: int):
        if not self.on:
            self.on = True
            print(f'Turning on the air conditioner: it\'s {temperature} degrees here!')
        
    def stop(self, temperature: int):
        if self.on:
            self.on = False
            print(f'Turning off the air conditioner: it\'s {temperature} degrees here!')
        

class DrinkMaker:
    
    def make_pina_colada(self, temperature : int):
        print(f'Making a pina colada: it\'s {temperature} degrees here!')
    
    def make_hot_chocolate(self, temperature: int):
        print(f'Making a hot chocolate: it\'s {temperature} degrees here!')
    
    
class Heating:
    
    def __init__(self, on: bool = False):
        self.on = on
    
    def start(self, temperature: int):
        if not self.on:
            self.on = True
            print(f'Turning on the heating: it\'s {temperature} degrees here!')
        
    def stop(self, temperature: int):
        if self.on: 
            self.on = False
            print(f'Turning off the heating: it\'s {temperature} degrees here!')


class FireFighters:
    
    def call(self, temperature: int):
        print(f'Calling the firefighters: it\'s {temperature} degrees here!')
        

class SmartThermostat:
    
    @staticmethod
    def create() -> 'SmartThermostat':
        smart_thermostat = SmartThermostat()
        smart_thermostat.fire_fighters = FireFighters()
        smart_thermostat.air_conditioner = AirConditioner()
        smart_thermostat.heating = Heating()
        smart_thermostat.drink_maker = DrinkMaker()
        return smart_thermostat
        
    
    def set_temperature(self, temperature: int):
        self.temperature = temperature
        
        if self.temperature > 50:
            self.fire_fighters.call(self.temperature)
            return
        
        if self.temperature < 20: 
            self.air_conditioner.stop(self.temperature)
            self.heating.start(self.temperature)
            self.drink_maker.make_hot_chocolate(self.temperature)
            
        if self.temperature > 30: 
            self.heating.stop(self.temperature)
            self.air_conditioner.start(self.temperature)
            self.drink_maker.make_pina_colada(self.temperature)



smart_thermostat = SmartThermostat.create()

smart_thermostat.set_temperature(15)

smart_thermostat.set_temperature(35)

smart_thermostat.set_temperature(9999)

Nosso termostato está funcionando, mas nao é difícil ver que nosso método `set_temperature` é bem pouco sustentável. Além disso, a classe `SmartThermostat` já tem um número considerável de dependências, que deve crescer mais ainda à medida que integremos novas funcionalidades ao nosso termostato. Vejamos como o _Observer_ resolveria o problema:

In [ ]:
import abc
from typing import Set

class Observer(abc.ABC):
    
    @abc.abstractmethod
    def update(self, temperature: int):
        pass

    
class AirConditioner(Observer):
    
    def __init__(self, on: bool = False):
        self.on = on
        
    def start(self, temperature: int):
        if not self.on:
            self.on = True
            print(f'Turning on the air conditioner: it\'s {temperature} degrees here!')
        
    def stop(self, temperature: int):
        if self.on:
            self.on = False
            print(f'Turning off the air conditioner: it\'s {temperature} degrees here!')
    
    def update(self, temperature: int):
        if temperature > 30:
            self.start(temperature)
        elif temperature < 20:
            self.stop(temperature)
        

class DrinkMaker(Observer):
    
    def make_pina_colada(self, temperature : int):
        print(f'Making a pina colada: it\'s {temperature} degrees here!')
    
    def make_hot_chocolate(self, temperature: int):
        print(f'Making a hot chocolate: it\'s {temperature} degrees here!')
        
    def update(self, temperature: int):
        if temperature > 30:
            self.make_pina_colada(temperature)
        elif temperature < 20:
            self.make_hot_chocolate(temperature)
    
    
class Heating(Observer):
    
    def __init__(self, on: bool = False):
        self.on = on
    
    def start(self, temperature: int):
        if not self.on:
            self.on = True
            print(f'Turning on the heating: it\'s {temperature} degrees here!')
        
    def stop(self, temperature: int):
        if self.on: 
            self.on = False
            print(f'Turning off the heating: it\'s {temperature} degrees here!')
            
    def update(self, temperature: int):
        if temperature > 30:
            self.stop(temperature)
        elif temperature < 20:
            self.start(temperature)


class FireFighters(Observer):
    
    def call(self, temperature: int):
        print(f'Calling the firefighters: it\'s {temperature} degrees here!')
        
    def update(self, temperature: int):
        if temperature > 50:
            self.call(temperature)
        

class SmartThermostat:
    
    def __init__(self, observers: Set[Observer] = None):
        self.observers = observers or set();
    
    def attach(self, observer: Observer):
        self.observers.add(observer)
        
    def detach(observer: Observer):
        self.observers.delete(observer)
        
    def notify(self):
        for observer in self.observers:
            observer.update(self.temperature)

    def set_temperature(self, temperature: int):
        self.temperature = temperature
        self.notify()



smart_thermostat = SmartThermostat()

for observer in [FireFighters(), Heating(), AirConditioner(), DrinkMaker()]:
    smart_thermostat.attach(observer)

smart_thermostat.set_temperature(15)

smart_thermostat.set_temperature(35)

smart_thermostat.set_temperature(9999)

Com a mudanca, nossa classe `SmartThermostat` está muito mais limpa e sustentável. Nosso termostato tem somente duas responsabilidades: guardar o valor da temperatura e informá-lo aos observers. Cabe aos observers implementar sua lógica de negócio de acordo com o estado do Subject. Isso fica evidenciado numa pequena mudanca de comportamento: no exemplo anterior, `SmartThermostat` nao invocava nenhum método das classes dependentes se a casa estivesse pegando fogo. A versao atual evidenciou um possível bug na nossa classe `DrinkMaker`: às vezes pode fazer calor demais para uma piña colada.

## Exemplo 2: MVC



O _Observer_ é uma peca-chave da arquitetura MVC (Model-View-Controller), usada para a criacao de interfaces gráficas. O Model é responsável pela lógica de negócio da aplicacao, enquanto a View oferece uma interface ao usuário e exibe informacoes sobre o estado do Model. Portanto, a relacao entre o Model e a View é uma relacao Subject-Observer. 

Cabe ao Controller o papel de servir como um intermediário entre a View e o Model, permitindo que o usuário interaja com o Model sem que a View e o Model estejam diretamente conectados. Nesse caso, pode-se dizer que o Controller exerce o papel de _Mediator_. O exemplo abaixo, um cronômetro, mostra a interacao entre essas classes.

In [2]:
import sys
!{sys.executable} -m pip install pynput --user

In [2]:
import abc
from typing import Set
import threading
from time import sleep
from pynput import keyboard


class Observer(abc.ABC):
    
    @abc.abstractmethod
    def update(self, seconds: int):
        pass


class ChronometerModel:
    
    def __init__(self, seconds: int = 0, on: bool = False, observers: Set[Observer] = None):
        self.seconds = seconds
        self.on = on
        self.observers = observers or set()
    
    def start(self):
        if not self.on:
            self.on = True
            self.thread = threading.Thread(target=self.tick, daemon=True)
            self.thread.start()
    
    def stop(self):
        if self.on:
            self.on = False
            self.thread.join()
            
    def reset(self):
        self.seconds = 0
        
    def tick(self):
        sleep(1)
        if self.on:
            self.seconds += 1
            self.notify()
            self.tick()

    def notify(self):
        for observer in self.observers:
            observer.update(self.seconds)
            
    def attach(self, observer: Observer):
        self.observers.add(observer)
        
    def detach(self, observer: Observer):
        self.observers.delete(observer)
    

class ChronometerView(Observer): 
    
    def __init__(self, controller: 'ChronometerController'):
        self.controller = controller
    
    def load(self):
        self.notify_user('Press 1 to start, 2 to stop, 0 to reset or esc to exit \n')
        self.listener = keyboard.Listener(on_press=self.on_press)
        self.listener.start()
        self.listener.join()
                      
    def update(self, seconds: int):
        print(seconds)
    
    def notify_user(self, message: str):
        print(message)
    
    def on_press(self, key):
        if key == keyboard.Key.esc:
            self.controller.stop()
            self.notify_user('Exiting')
            return False
        try:
            k = key.char
        except:
            k = key.name
        if k == '1': 
            self.controller.start()
        if k == '2':
            self.controller.stop()
        if k == '0':
            self.controller.reset()
    

class ChronometerController:
    
    def __init__(self, model: ChronometerModel = None):
        self.model = model or ChronometerModel()
        self.view = ChronometerView(self)
        self.model.attach(self.view)
    
    def load_view(self):
        self.view.load()

    def start(self):
        self.view.notify_user('Starting chronometer')
        self.model.start()
    
    def stop(self):
        self.model.stop()
        self.view.notify_user('Stopping chronometer')
    
    def reset(self):
        self.view.notify_user('Resetting chronometer')
        self.model.reset()

ch = ChronometerController()
ch.load_view()

Press 1 to start, 2 to stop, 0 to reset or esc to exit 

Starting chronometer
1
2
3
Stopping chronometer
Stopping chronometer
Exiting


## Prós e contras:

### Prós

- Simplifica o código da classe observada (Subject)
- Promove o _decoupling_ entre Observer e Subject
- Permite a criacao de relacoes entre os objetos durante o runtime

### Contras:

- Pode gerar notificacoes irrelevantes a determinados Observers
- Nao há controle sobre a ordem em que os Observers sao notificados


## Discussao:

Part 1: The classic Model-View-Controller design is explained in Implementation note #8: Encapsulating complex update semantics. Would it ever make sense for an Observer (or View) to talk directly to the Subject (or Model)?

Part 2: What are the properties of a system that uses the Observer pattern extensively? How would you approach the task of debugging code in such a system?

Part 3: Is it clear to you how you would handle concurrency problems with is pattern? Consider an Unregister() message being sent to a subject, just before the subject sends a Notify() message to the ChangeManager (or Controller).
